In [ ]:
from flask import Flask, request, jsonify
import cv2
from deepface import DeepFace
import numpy as np
from flask_cors import CORS  # Import the CORS extension


app = Flask(__name__)
CORS(app)

@app.route('/upload_photo', methods=['POST'])
def predict_emotion():
    print("Inside Predict Emotion")
    try:
        # print("hi ", request.files)
        # Check if the POST request has the file part
        # if 'image' not in request.files['photo']:
        #     print("No Image File Found!")
        #     return jsonify({'error': 'No image file in the request'}), 400

        # Get the image file from the request
        file = request.files["photo"]
        print(file)

        # Read the image from the file
        img_stream = file.read()
        img_array = bytearray(img_stream)
        np_img = np.asarray(img_array, dtype=np.uint8)
        frame = cv2.imdecode(np_img, 1)

        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(gray,1.1,4)

        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        emotion = result[0]['dominant_emotion']
    
        df = pd.read_csv('Downloads/SpotifySongs.csv')
        df.drop(columns = ['Duration_ms', 'Popularity'], inplace = True)
        df.isnull().sum()
        df.dropna(inplace = True)
        df.duplicated().sum()
        df = df.drop_duplicates()

        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df.iloc[:, 2:])
        df_scaled = pd.DataFrame(df_scaled, columns=df.columns[2:])
        df_scaled = pd.concat([df.iloc[:, :2], df_scaled], axis=1)
        df_scaled = df_scaled.drop_duplicates()
        df_scaled.duplicated().sum()
        df_scaled.dropna(inplace = True)

        happy_song_features = [[0.8, 0.7, 0.0, 0.7, 0.0, 0.5, 0.1, 0.1, 0.2, 0.5, 120]]
        sad_song_features = [[0.4, 0.4, 0.5, 0.4, 0.5, 0.3, 0.05, 0.05, 0.05, 0.3, 100]]
        angry_song_features = [[0.5, 0.9, 0.0, 0.8, 0.5, 0.5, 0.15, 0.35, 0.15, 0.4, 140]]
        neutral_song_features = [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1, 0.5, 110]]

        a = scaler.transform(happy_song_features)
        b = scaler.transform(sad_song_features)
        c = scaler.transform(angry_song_features)
        d = scaler.transform(neutral_song_features)

        if (emotion=='happy'):
            x = np.array(a)
        elif (emotion=='sad'):
            x = np.array(b)
        elif (emotion=='angry'):
            x = np.array(c)
        else:
            x = np.array(d)

        X = df_scaled.drop(columns=['SongName', 'ArtistName'])
        cosine_similarities = cosine_similarity(x, X)
        similar_song_indices = cosine_similarities.argsort()[0][::-1]

        similar_songs = df_scaled.iloc[similar_song_indices[:50]]['SongName'].tolist()
        
        return jsonify({'emotion': emotion, 'similar_songs': similar_songs})

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
from flask import Flask, jsonify, request
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from deepface import DeepFace
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/upload_photo', methods=['POST'])

def get_songs():
    try:
        # Receive image from the request
        file = request.files['photo']
        img_stream = file.read()
        img_array = bytearray(img_stream)
        np_img = np.asarray(img_array, dtype=np.uint8)
        frame = cv2.imdecode(np_img, 1)

        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        emotion = result[0]['dominant_emotion']

        # Load song data
        df = pd.read_csv('Downloads/SpotifySongs.csv')
        df.drop(columns=['Duration_ms', 'Popularity'], inplace=True)
        df.isnull().sum()
        df.dropna(inplace=True)
        df = df.drop_duplicates()

        # Scale song features
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df.iloc[:, 2:])
        df_scaled = pd.DataFrame(df_scaled, columns=df.columns[2:])
        df_scaled = pd.concat([df.iloc[:, :2], df_scaled], axis=1)
        df_scaled = df_scaled.drop_duplicates()
        df_scaled.dropna(inplace=True)

        # Define song features for each emotion
        happy_song_features = [[0.8, 0.7, 0.0, 0.7, 0.0, 0.5, 0.1, 0.1, 0.2, 0.5, 120]]
        sad_song_features = [[0.4, 0.4, 0.5, 0.4, 0.5, 0.3, 0.05, 0.05, 0.05, 0.3, 100]]
        angry_song_features = [[0.5, 0.9, 0.0, 0.8, 0.5, 0.5, 0.15, 0.35, 0.15, 0.4, 140]]
        neutral_song_features = [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1, 0.5, 110]]

        # Transform features based on detected emotion
        if emotion == 'happy':
            x = scaler.transform(happy_song_features)
        elif emotion == 'sad':
            x = scaler.transform(sad_song_features)
        elif emotion == 'angry':
            x = scaler.transform(angry_song_features)
        else:
            x = scaler.transform(neutral_song_features)

        cosine_similarities = cosine_similarity(x, df_scaled.drop(columns=['SongName', 'ArtistName']))
        similar_song_indices = cosine_similarities.argsort()[0][::-1]

        # Get top 50 similar songs
        similar_songs = df_scaled.iloc[similar_song_indices[:50]]['SongName'].tolist()

        return jsonify({'emotion': emotion, 'similar_songs': similar_songs})

    except Exception as e:
        return jsonify({'error': str(e)})


if __name__ == '__main__':
    app.run(debug=True, port=5000)

In [8]:
import cv2
from deepface import DeepFace
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

frame = cv2.imread('Desktop/hackdata/download.jpeg')

faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(gray,1.1,4)
for x,y,w,h in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    cv2. rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if len(facess) == 0:
        print("Face not detected")
    else:
        for (ex,ey,ew,eh) in facess:
            face_roi = roi_color[ey: ey+eh, ex:ex + ew]
            
result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
emotion = result[0]['dominant_emotion']
emotion

'happy'

In [24]:
df = pd.read_csv('Downloads/SpotifySongs.csv')
df.drop(columns = ['Duration_ms', 'Popularity'], inplace = True)
df.isnull().sum()
df.dropna(inplace = True)
df.duplicated().sum()
df = df.drop_duplicates()

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.iloc[:, 2:])
df_scaled = pd.DataFrame(df_scaled, columns=df.columns[2:])
df_scaled = pd.concat([df.iloc[:, :2], df_scaled], axis=1)
df_scaled = df_scaled.drop_duplicates()
df_scaled.duplicated().sum()
df_scaled.dropna(inplace = True)

happy_song_features = [[0.8, 0.7, 0.0, 0.7, 0.0, 0.5, 0.1, 0.1, 0.2, 0.5, 120]]
sad_song_features = [[0.4, 0.4, 0.5, 0.4, 0.5, 0.3, 0.05, 0.05, 0.05, 0.3, 100]]
angry_song_features = [[0.5, 0.9, 0.0, 0.8, 0.5, 0.5, 0.15, 0.35, 0.15, 0.4, 140]]
neutral_song_features = [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1, 0.5, 110]]

a = scaler.transform(happy_song_features)
b = scaler.transform(sad_song_features)
c = scaler.transform(angry_song_features)
d = scaler.transform(neutral_song_features)

if (emotion=='happy'):
    x = np.array(a)
elif (emotion=='sad'):
    x = np.array(b)
elif (emotion=='angry'):
    x = np.array(c)
else:
    x = np.array(d)

X = df_scaled.drop(columns=['SongName', 'ArtistName'])
cosine_similarities = cosine_similarity(x, X)
similar_song_indices = cosine_similarities.argsort()[0][::-1]

similar_songs = df_scaled.iloc[similar_song_indices[:50]]['SongName'].tolist()
print(similar_songs)

['Lost', 'Million Ways', 'Kiss Me More (feat. SZA)', 'Before You Go', 'Fall', 'Hands To Myself', 'good 4 u', "What's Up", 'Hope', 'Old Friends', 'Rockstar', 'Hold On', 'Astronaut In The Ocean', 'The One I Love', 'Say My Name', 'When I Need You', 'Share', 'Dharala Prabhu Title Track (From "Dharala Prabhu")', 'Drag Me Down', 'Saiyaan Ji', 'Down Below', 'Short Change Hero', 'Psycho (feat. Ty Dolla $ign)', 'Just Like You', 'Go Crazy', 'Dip (feat. Nicki Minaj) (feat. Nicki Minaj)', 'Sunflower - Spider-Man: Into the Spider-Verse', 'The Angel and the Fool', "Hollywood's Bleeding", 'People Watching', 'Lucid Dreams (feat. Lil Uzi Vert) - Remix', 'Turks (with Gunna & ft. Travis Scott)', 'Lemonade - Remix', 'White Flag', 'Waves', 'exile (feat. Bon Iver)', 'Main Title', 'exile', 'wannabe', 'Need To Know', 'Moon Rider (feat. Wrabel)', 'In Your Arms (with X Ambassadors)', 'Roses', 'All Night', 'Leave Before You Love Me (with Jonas Brothers)', 'Paris', 'SOS (feat. Aloe Blacc)', 'Closer', 'Taki Taki (

In [9]:
df = pd.read_csv('Downloads/SpotifySongs.csv')
df.drop(columns = ['Duration_ms', 'Popularity'], inplace = True)
df.isnull().sum()
df.dropna(inplace = True)
df.duplicated().sum()
df = df.drop_duplicates()

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df.iloc[:, 2:])
df_scaled = pd.DataFrame(df_scaled, columns=df.columns[2:])
df_scaled = pd.concat([df.iloc[:, :2], df_scaled], axis=1)
df_scaled = df_scaled.drop_duplicates()
df_scaled.duplicated().sum()
df_scaled.dropna(inplace = True)

happy_song_features = [[0.8, 0.7, 0.0, 0.7, 0.0, 0.5, 0.1, 0.1, 0.2, 0.5, 120]]
sad_song_features = [[0.4, 0.4, 0.5, 0.4, 0.5, 0.3, 0.05, 0.05, 0.05, 0.3, 100]]
angry_song_features = [[0.5, 0.9, 0.0, 0.8, 0.5, 0.5, 0.15, 0.35, 0.15, 0.4, 140]]
neutral_song_features = [[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.1, 0.1, 0.5, 110]]

a = scaler.transform(happy_song_features)
b = scaler.transform(sad_song_features)
c = scaler.transform(angry_song_features)
d = scaler.transform(neutral_song_features)

if (emotion=='happy'):
    x = np.array([a])
elif (emotion=='sad'):
    x = np.array([b])
elif (emotion=='angry'):
    x = np.array([c])
else:
    x = np.array([d])

x_1d = x.flatten()
X = df_scaled.drop(columns=['SongName', 'ArtistName']).values

num_features = len(X[0])
annoy_index = AnnoyIndex(num_features, 'angular')

for i, features in enumerate(X):
    annoy_index.add_item(i, features)

annoy_index.build(n_trees=10)
similar_song_indices = annoy_index.get_nns_by_vector(x_1d, n=31, include_distances=False)
similar_songs = df_scaled.iloc[similar_song_indices[1:]]['SongName'].tolist()
print(len(similar_songs))

['Million Ways', 'Kiss Me More (feat. SZA)', 'Hope', 'Hands To Myself', 'good 4 u', "What's Up", 'Before You Go', 'Fall', 'Old Friends', 'Rockstar', 'Hold On', 'Share', 'Drag Me Down', 'Saiyaan Ji', 'exile (feat. Bon Iver)', 'White Flag', 'Lemonade - Remix', 'Turks (with Gunna & ft. Travis Scott)', 'Lucid Dreams (feat. Lil Uzi Vert) - Remix', 'People Watching', "Hollywood's Bleeding", 'Just Like You', 'Sunflower - Spider-Man: Into the Spider-Verse', 'Dip (feat. Nicki Minaj) (feat. Nicki Minaj)', 'Go Crazy', 'Down Below', 'Psycho (feat. Ty Dolla $ign)', 'Short Change Hero', 'Waves', 'The Angel and the Fool']


In [3]:
from flask import Flask, request, jsonify
import cv2
from deepface import DeepFace

app = Flask(__name__)

@app.route('/predict_emotion', methods=['POST'])
def predict_emotion():
    try:
        # Get the image file from the request
        file = request.files['image']

        # Read the image from the file
        img_stream = file.read()
        img_array = bytearray(img_stream)
        np_img = np.asarray(img_array, dtype=np.uint8)
        frame = cv2.imdecode(np_img, 1)

        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(gray,1.1,4)
        
        for x,y,w,h in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2. rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            facess = faceCascade.detectMultiScale(roi_gray)
            if len(facess) == 0:
                print("Face not detected")
            else:
                for (ex,ey,ew,eh) in facess:
                    face_roi = roi_color[ey: ey+eh, ex:ex + ew]

        result = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        emotion = result[0]['dominant_emotion']

        return jsonify({'emotion': emotion})

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/yagaykhatri/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/yagaykhatri/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/yagaykhatri/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/yagaykhatri/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 991, in launch_instance
    app.initialize(argv)
  File "/Users/yagaykhatri/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/Users/yagaykhatri/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 665, in initialize
    self.init_sockets()
  File "/Users/yagay

SystemExit: 1